In [ ]:
import random
import math
import numpy as np


class Soldier:
    def __init__(self):
        self.position = None
        self.damage = 0
        self.fitness = 0


def distancia_euclidiana(soldier1, soldier2):
    x1, y1, z1 = soldier1.position
    x2, y2, z2 = soldier2.position
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)


def rosenbrock(x):
    N = len(x)
    sum = 0
    for i in range(N - 1):
        sum += 100 * (x[i + 1] - x[i]**2)**2 + (1 - x[i])**2
    return sum


def create_random_position():
    # Create random position within bounds
    position = []
    for i in range(dimensions):
        position.append(random.uniform(search_space[0], search_space[1]))
    return position


def create_random_soldier():
    # Create a random soldier
    soldier = Soldier()
    soldier.position = create_random_position()
    soldier.damage = 0
    soldier.fitness = 0
    return soldier


def init_population(size):
    # Initialize random population of soldiers
    population = []
    for i in range(size):
        soldier = create_random_soldier()
        population.append(soldier)
    return population

def calculate_best_global(population):
    # Supongamos que el mejor global es el soldado con el mayor fitness
    best_soldier = max(population, key=lambda soldier: soldier.fitness)
    return best_soldier


def calculate_fitness(soldier):
    # Calculate fitness of a soldier
    return rosenbrock(soldier.position)


def evaluate_soldiers():
    # Evaluate fitness of all soldiers
    for soldier in population:
        soldier.fitness = calculate_fitness(soldier)


def find_closest(soldier, population):
    min_distance = float('inf')  # Inicializamos con una distancia infinita
    closest_soldier = None

    for other_soldier in population:
        if other_soldier != soldier:  # Evitar comparar el soldado consigo mismo
            distance = distancia_euclidiana(soldier, other_soldier)
            if distance < min_distance:
                min_distance = distance
                closest_soldier = other_soldier

    return closest_soldier


def fight_enemies():

    # Each soldier fights nearest neighbor
    for soldier in population:
        closest = find_closest(soldier, population)
        if soldier.fitness > closest.fitness:
            closest.damage += 1
            damaged_soldiers.append(closest)
        else:
            victoriuos_soldiers.append(soldier)
    return damaged_soldiers, victoriuos_soldiers

def respawn_dead(damaged_soldiers, lower_bounds, upper_bounds):
    r = random.uniform(0, 1)
    # Soldiers dead from too much damage respawn
    for soldier in damaged_soldiers:
        if soldier.damage > threshold:
            soldier.position = r(upper_bounds - lower_bounds) + lower_bounds #Cambiar a una posicion segun la zona factible 
            soldier.damage = 0


def move_damaged(damaged_soldiers, elite):
    r = random.uniform(0, 1)
    # Damaged soldiers move towards elite
    for soldier in damaged_soldiers:
        soldier.position = soldier.position + r * (elite.position - soldier.position)

def calculate_dynamic_bounds(elite):
    # Calcular límites dinámicos
    lower_bounds = elite.position - np.std([soldier.position for soldier in damaged_soldiers], axis=0)
    upper_bounds = elite.position + np.std([soldier.position for soldier in damaged_soldiers], axis=0)
    return lower_bounds, upper_bounds




#Inicializacion de parametros

search_space = [(-2, 2), (-2, 2), (-2, 2)]
size = 100
elite = None
dimensions = 3
max_iter = 100
shrink_rate = 10#Cambiar a un calculo delta 
population = init_population(size)
threshold = 3
damaged_soldiers = []
victoriuos_soldiers = []

for i in range(max_iter):
    evaluate_soldiers()
    
    fight_enemies()
    
    respawn_dead() 
    
    move_damaged()
    
    elite = calculate_best_global()
    
    if i % shrink_rate == 0:
        calculate_dynamic_bounds(elite)
        
best_solution = elite